# Batch 1: Agentic Text2Cypher Inference (Day 1/5)

**Configurations**: Zero-Shot_Full, Zero-Shot_Nodes+Paths

**Model**: Qwen3-32B via Groq API

**Questions**: 52 × 2 = 104 inferences

---

## Rate Limit Strategy
- Groq API: 500K tokens/day
- Estimated tokens per inference: ~5000 (with retries)
- Safe limit: ~100 inferences/day
- This batch: 104 inferences (should fit within limit)

In [1]:
# Setup
import sys
import os
from pathlib import Path

# Set project root
project_root = Path.cwd().parent
os.chdir(project_root)

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Working directory: {os.getcwd()}")

Project root: /Users/tsimiscouse/Docs/Sarjana/Skripsi/kg-luthfi/agentic
Working directory: /Users/tsimiscouse/Docs/Sarjana/Skripsi/kg-luthfi/agentic


In [2]:
# Imports
import json
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm

from config.settings import Settings
from config.llm_config import LLMConfig
from data.ground_truth_loader import GroundTruthLoader
from prompts.prompt_manager import PromptManager, PromptType, SchemaFormat
from experiment.batch_processor import BatchProcessor

In [3]:
# Initialize settings
settings = Settings()
llm_config = LLMConfig()

print(f"LLM Provider: {llm_config.provider}")
print(f"Model: {llm_config.model}")
print(f"Max Iterations: {settings.max_iterations}")

LLM Provider: openrouter
Model: qwen/qwen-2.5-coder-32b-instruct
Max Iterations: 3


In [4]:
# Validate API Key
try:
    llm_config.validate()
    print("API Key validated successfully!")
except ValueError as e:
    print(f"Error: {e}")

API Key validated successfully!


In [5]:
# Load ground truth data
loader = GroundTruthLoader()
items = loader.load()
print(f"Loaded {len(items)} questions")

Loaded 52 questions


## Batch 1 Configurations

| # | Prompt Type | Schema Format | Config Name |
|---|-------------|---------------|-------------|
| 1 | Zero-Shot | Full Schema | Zero-Shot_Full |
| 2 | Zero-Shot | Nodes+Paths | Zero-Shot_Nodes+Paths |

In [6]:
# Define Batch 1 configurations
BATCH_1_CONFIGS = [
    {"prompt": PromptType.ZERO_SHOT, "schema": SchemaFormat.FULL_SCHEMA},
    {"prompt": PromptType.ZERO_SHOT, "schema": SchemaFormat.NODES_PATHS},
]

print(f"Batch 1: {len(BATCH_1_CONFIGS)} configurations")
print(f"Total inferences: {len(BATCH_1_CONFIGS) * len(items)} = {len(BATCH_1_CONFIGS)} × {len(items)}")

Batch 1: 2 configurations
Total inferences: 104 = 2 × 52


In [7]:
# Setup results directory
results_dir = project_root / "results_v2"
results_dir.mkdir(parents=True, exist_ok=True)

batch_results_dir = results_dir / "batch_1"
batch_results_dir.mkdir(parents=True, exist_ok=True)

print(f"Results will be saved to: {batch_results_dir}")

Results will be saved to: /Users/tsimiscouse/Docs/Sarjana/Skripsi/kg-luthfi/agentic/results_v2/batch_1


In [8]:
# Initialize batch processor
processor = BatchProcessor(
    settings=settings,
    llm_config=llm_config,
    checkpoint_dir=str(batch_results_dir / "checkpoints")
)

prompt_manager = PromptManager()

## Run Batch 1 Experiments

In [ ]:
# Run experiments for Batch 1
batch_1_results = {}

for i, config in enumerate(BATCH_1_CONFIGS):
    config_name = prompt_manager.get_configuration_name(config["prompt"], config["schema"])
    print(f"\n{'='*60}")
    print(f"Configuration {i+1}/{len(BATCH_1_CONFIGS)}: {config_name}")
    print(f"{'='*60}")
    
    # Create config directory
    config_dir = batch_results_dir / config_name
    config_dir.mkdir(parents=True, exist_ok=True)
    
    # Process batch
    try:
        results = processor.process_batch(
            items=items,
            prompt_type=config["prompt"],
            schema_format=config["schema"],
            batch_id=f"batch1_{config_name}",
            resume=True
        )
        
        batch_1_results[config_name] = results
        
        # Save results
        results_data = [state.to_dict() for state in results]
        
        with open(config_dir / "agentic_results.json", "w") as f:
            json.dump(results_data, f, indent=2, default=str)
        
        # Save as CSV
        df = pd.DataFrame([{
            "question_id": s.question_id,
            "question": s.question,
            "ground_truth": s.ground_truth_query,
            "final_query": s.final_query,
            "success": s.success,
            "pass_at_1": s.first_attempt_success,
            "total_iterations": s.total_iterations,
            "kg_valid": s.kg_valid,
        } for s in results])
        df.to_csv(config_dir / "agentic_results.csv", index=False)
        
        # Quick stats
        success_rate = sum(1 for s in results if s.success) / len(results) * 100
        pass_at_1 = sum(1 for s in results if s.first_attempt_success) / len(results) * 100
        avg_iterations = sum(s.total_iterations for s in results) / len(results)
        
        print(f"\n{config_name} Results:")
        print(f"  Success Rate: {success_rate:.1f}%")
        print(f"  Pass@1 Rate: {pass_at_1:.1f}%")
        print(f"  Avg Iterations: {avg_iterations:.2f}")
        
    except Exception as e:
        print(f"Error processing {config_name}: {e}")
        import traceback
        traceback.print_exc()


Configuration 1/2: Zero-Shot_Full


## Batch 1 Summary

In [ ]:
# Generate batch summary
if batch_1_results:
    summary_rows = []
    
    for config_name, results in batch_1_results.items():
        success_count = sum(1 for s in results if s.success)
        pass_at_1_count = sum(1 for s in results if s.first_attempt_success)
        kg_valid_count = sum(1 for s in results if s.kg_valid)
        
        summary_rows.append({
            "Configuration": config_name,
            "Total": len(results),
            "Success": success_count,
            "Success Rate (%)": round(success_count / len(results) * 100, 2),
            "Pass@1": pass_at_1_count,
            "Pass@1 Rate (%)": round(pass_at_1_count / len(results) * 100, 2),
            "KG Valid": kg_valid_count,
            "KG Valid Rate (%)": round(kg_valid_count / len(results) * 100, 2),
            "Avg Iterations": round(sum(s.total_iterations for s in results) / len(results), 2),
        })
    
    df_summary = pd.DataFrame(summary_rows)
    display(df_summary)
    
    # Save summary
    df_summary.to_csv(batch_results_dir / "batch_1_summary.csv", index=False)
    print(f"\nSummary saved to: {batch_results_dir / 'batch_1_summary.csv'}")

In [ ]:
# Save batch metadata
batch_metadata = {
    "batch_number": 1,
    "date": datetime.now().isoformat(),
    "configurations": [prompt_manager.get_configuration_name(c["prompt"], c["schema"]) for c in BATCH_1_CONFIGS],
    "total_questions": len(items),
    "total_inferences": len(BATCH_1_CONFIGS) * len(items),
    "llm_model": llm_config.model,
    "max_iterations": settings.max_iterations,
}

with open(batch_results_dir / "batch_1_metadata.json", "w") as f:
    json.dump(batch_metadata, f, indent=2)

print("Batch 1 completed!")
print(f"Continue with batch-2_01_agentic_inference.ipynb tomorrow.")

In [ ]:
# Cleanup
processor.close()